<a href="https://colab.research.google.com/github/hfelizzola/Investigaciones-de-Operaciones-I/blob/main/flujo-redes/03_modelo_transporte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de Transporte

In [8]:
!pip install gamspy --quiet

## Problema de Distribución

Uno de los productos más importantes de la P & T COMPANY es el chícharo enlatado. Los chicharos se preparan en tres enlatadoras:  Bellingham, Eugene y Albert Lea.  Después se envían por camión a cuatro almacenes de distribución: Sacramento, Salt Lake City, Rapid City y Albuquerque. Debido a que los costos de transporte constituyen un gasto importante, la administración ha iniciado un estudio para reducirlos a su mínima expresión. Se ha estimado la capacidad de producción de cada enlatadora durante la próxima temporada y se ha asignado a cada almacén cierta cantidad de la producción total de chicharos, esto dependiente la demanda de la zona de influencia.

<table class="w-full">
    <thead>
        <tr>
            <th class="rounded-tl-xl"></th>
            <th>Almacén 1</th>
            <th>Almacén 2</th>
            <th>Almacén 3</th>
            <th>Almacén 4</th>
            <th class="rounded-tr-xl">Producción</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td class="font-bold">Embaladora 1</td>
            <td>464</td>
            <td>513</td>
            <td>654</td>
            <td>867</td>
            <td>75</td>
        </tr>
        <tr>
            <td class="font-bold">Embaladora 2</td>
            <td>352</td>
            <td>416</td>
            <td>690</td>
            <td>791</td>
            <td>125</td>
        </tr>
        <tr>
            <td class="font-bold">Embaladora 3</td>
            <td class="rounded-bl-xl">995</td>
            <td>682</td>
            <td>588</td>
            <td>685</td>
            <td>100</td>
        </tr>
        <tr>
            <td class="font-bold">Asignación</td>
            <td>80</td>
            <td>65</td>
            <td>70</td>
            <td>85</td>
            <td></td>
        </tr>
    </tbody>
</table>

**Modelo Matemático**

- Conjuntos:
  - $i$: Nodos de origen, $I = \{1,2,3\}$
  - $j$: Nodos de destino, $J = \{1,2,3\}$
- Parámetros:
  - $c_{ij}$: Costo de transporte de $i \to j$
  - $s_i$: Capacidad del nodo $i$
  - $d_j$: Demanda del nodo $j$
- Variable de decisión:
  - $x_{ij}$: Cantidad a enviar de $i \to j$
- Función Objetivo y Restricciones:

\begin{equation*}
\text{Min. } Z = \sum_{i=1}^{m}\sum_{j=1}^{n} c_{ij}x_{ij}
\end{equation*}
\begin{align*}
SA:\\
\sum_{j=1}^{n} x_{ij} &= s_i && \forall i \in I\\
\sum_{i=1}^{m} x_{ij} &= d_j && \forall j \in J\\
x_{ij} &\ge 0, && \forall i \in I, \forall j \in J\\\\
\end{align*}

In [9]:
import numpy as np
import pandas as pd
from gamspy import Container, Set, Variable, Parameter, Equation, Model, Sense, Sum, Options

In [10]:
m = Container()

# Conjuntos
planta = Set(m, name='planta', records=['P1','P2','P3'])
almacen = Set(m, name='almacen', records=['A1','A2','A3','A4'])

# Variable de decisión
x = Variable(m, name='x', type='Positive', domain=[planta,almacen])

# Parametros
capacidad = Parameter(m, name='capacidad', domain=[planta], records=np.array([75,125,100]))
demanda = Parameter(m, name='demanda', domain=[almacen], records=np.array([80,65,70,85]))
costo = Parameter(m, name='costo', domain=[planta,almacen], records=np.array([[464,513,654,867],
                                                                              [352,416,690,791],
                                                                              [995,682,388,685]]))

# Restricciones
capacidad_max = Equation(m, name='capacidad_max', domain=[planta])
capacidad_max[planta] = Sum(almacen, x[planta,almacen]) == capacidad[planta]

demanda_max = Equation(m, name='demanda_max', domain=[almacen])
demanda_max[almacen] = Sum(planta, x[planta,almacen]) == demanda[almacen]

costo_total = Sum([planta,almacen], costo[planta,almacen] * x[planta,almacen])

# Model
modelo_transporte = Model(m, name='modelo_transporte', objective=costo_total, sense=Sense.MIN, problem='LP', equations=[capacidad_max, demanda_max])
modelo_transporte.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,152535.0,8,13,LP,CPLEX,0.001


In [11]:
x.records.pivot(index="planta", columns="almacen", values='level')

almacen,A1,A2,A3,A4
planta,,,,
P1,0.0,20.0,0.0,55.0
P2,80.0,45.0,0.0,0.0
P3,0.0,0.0,70.0,30.0


In [12]:
capacidad_max.records

,planta,level,marginal,lower,upper,scale
0,P1,75.0,97.0,75.0,75.0,1.0
1,P2,125.0,0.0,125.0,125.0,1.0
2,P3,100.0,-85.0,100.0,100.0,1.0


In [13]:
demanda_max.records

,almacen,level,marginal,lower,upper,scale
0,A1,80.0,352.0,80.0,80.0,1.0
1,A2,65.0,416.0,65.0,65.0,1.0
2,A3,70.0,473.0,70.0,70.0,1.0
3,A4,85.0,770.0,85.0,85.0,1.0
